In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from sklearn import model_selection, svm
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
import data_process_funcs as dpf
import ml_funcs as mlf

# Combos

Experiment Type

| Train         | Test          |               |               |
| ------------- | ------------- | ------------- | ------------- |
| Full Hardware | Sim           |R Sim          | Sim +R Sim    |
| Sim           | R Sim         |Full Hardware  |               |
| R Sim         | Sim           |Full Hardware  |               |
| Sim +R Sim    |Full Hardware  |               |               |

Hardware age
| Train         | Test          |
| ------------- | ------------- |
| New Hardware  |Old Hardware   |
| Old Hardware  |New Hardware   |

Circuit type

| Train         | Test          |               |               |
| ------------- | ------------- | ------------- | ------------- |
| c1            | c2            |c3             | c2,c3         |
| c2            | c1            |c3             | c1,c3         |
| c3            | c1            |c1             | c1,c2         |
| c1,c2         | c3            |               |               |
| c1,c3         | c2            |               |               |
| c2,c3         | c1            |               |               |




In [92]:
def make_same_backends(dfs,backends):
    dfs_ = dfs.copy()
    dfs_mod = []
    for df in dfs_:
        df = df[df['backend'].isin(backends)]
        dfs_mod.append(df)
    
    return dfs_mod

def get_HSR_array_all_backends(nr_qubits):

    df_H = dpf.get_expanded_df('Hardware',nr_qubits)
    df_S = dpf.get_expanded_df('Simulation',nr_qubits)
    df_R = dpf.get_expanded_df('Refreshed_Simulation',nr_qubits)

    return [df_H, df_S, df_R]

def get_circuit_type_array(df_nq):
    df_nqi = mlf.features_to_int(df_nq)
    circuits = mlf.split_into_circuits(df_nqi) 
    return circuits

def get_HSR_list_of_arrays(nr_qubits):
    initial_list = get_HSR_array_all_backends(nr_qubits)
    list_of_arrays = generate_combos(initial_list)
    
    df_SR = pd.concat(initial_list[1:3].copy())
    #print(df_SR)
    # Quick and dirty fix for results_to_csv function:
    df_SR['experiment_type'] = 'Sim and Refreshed'

    #Train on H, Test on SR combined:
    list_of_arrays[0].append(df_SR)

    #make the train on H row only torino and brisbane:
    #H_backends = initial_list[0]['backend'].unique()
    for i in range(len(list_of_arrays)):
        backends = initial_list[i]['backend'].unique()
        list_of_arrays[i] = make_same_backends(list_of_arrays[i],backends)
    
    #Train on SR and Test on H only:
    train_SR_test_H = [df_SR,initial_list[0]]
    list_of_arrays.append(train_SR_test_H)

    return list_of_arrays

def get_circuits_list_of_arrays(df_nq):
    circuits = get_circuit_type_array(df_nq)
    combos = mlf.generate_combos(circuits,True)

    # add the combined training rows
    for i in len(circuits):
        combined_train = [combos[i][4],combos[i][0]]
        combos.append(combined_train)
    
    return combos

def generate_combos(individual_dfps,include_combined=False):
    indiv_list = individual_dfps.copy()
    nr_indiv = len(indiv_list)
    # print('indiv_dfps', individual_dfps)
    table =[]
    table.append(indiv_list)
    # print('first combo',table[0])
    # print('indiv_dfps', individual_dfps)

    for i in range(1,nr_indiv):
        #print(i)
        row = indiv_list.copy()
        # print('indiv_dfps', individual_dfps)
        # print('combos previous',table)
        train_element =  row.pop(i)
        row.insert(0,train_element)
        # print('i = ',i,row)
        # print('indiv_dfps', individual_dfps)
        
        if include_combined:
            #make elements joined as pairs
            pair_dfs = mlf.make_pairs(row[1:])
            #append the paired elements
            row = row+ pair_dfs
        
        table.append(row)
        #print('row added to combos',combos[i])
        # print('combos updated:',table)

    return table



In [80]:
initial_list = get_HSR_array_all_backends(4)


In [89]:
df_H =initial_list[0].copy()
H_backends = df_H['backend'].unique()

dfs_ = initial_list.copy()
dfs_mod =[]
for df in dfs_:
    df = df[df['backend'].isin(H_backends)]
    print('\ndf')
    print(df['experiment_type'].unique())
    print(df['backend'].unique())
    dfs_mod.append(df)

#all_bris_tor = make_same_backends(initial_list.copy(),['torino','brisbane'])


df
['Hardware']
['torino' 'brisbane']

df
['Simulation']
['torino' 'brisbane']

df
['Refreshed_Simulation']
['torino' 'brisbane']


In [86]:
for df in dfs_mod:
    print('\ndf')
    print(df['experiment_type'].unique())
    print(df['backend'].unique())


df
['Hardware']
['torino' 'brisbane']

df
['Simulation']
['torino' 'brisbane']

df
['Refreshed_Simulation']
['torino' 'brisbane']


In [48]:

list_of_letters = generate_combos(['a','b','c'])
print(list_of_letters)



indiv_dfps ['a', 'b', 'c']
first combo ['a', 'b', 'c']
indiv_dfps ['a', 'b', 'c']
indiv_dfps ['a', 'b', 'c']
combos previous [['a', 'b', 'c']]
i =  1 ['b', 'a', 'c']
indiv_dfps ['a', 'b', 'c']
combos updated: [['a', 'b', 'c'], ['b', 'a', 'c']]
indiv_dfps ['a', 'b', 'c']
combos previous [['a', 'b', 'c'], ['b', 'a', 'c']]
i =  2 ['c', 'a', 'b']
indiv_dfps ['a', 'b', 'c']
combos updated: [['a', 'b', 'c'], ['b', 'a', 'c'], ['c', 'a', 'b']]
[['a', 'b', 'c'], ['b', 'a', 'c'], ['c', 'a', 'b']]


In [59]:
for df in initial_list:
    print('df\n')
    print(df['experiment_type'].unique())

df

['Hardware']
df

['Simulation']
df

['Refreshed_Simulation']


In [63]:
test_table = generate_combos(initial_list)

for row in test_table:
    print("row\n")
    for df in row:
        print('df\n')
        print(df['experiment_type'].unique())
        print(df['backend'].unique())

row

df

['Hardware']
['torino' 'brisbane']
df

['Simulation']
['torino' 'fez' 'marrakesh' 'brisbane']
df

['Refreshed_Simulation']
['torino' 'brisbane']
row

df

['Simulation']
['torino' 'fez' 'marrakesh' 'brisbane']
df

['Hardware']
['torino' 'brisbane']
df

['Refreshed_Simulation']
['torino' 'brisbane']
row

df

['Refreshed_Simulation']
['torino' 'brisbane']
df

['Hardware']
['torino' 'brisbane']
df

['Simulation']
['torino' 'fez' 'marrakesh' 'brisbane']


In [93]:
nr_qubits = 4
combos_HSR = get_HSR_list_of_arrays(nr_qubits)

for row in combos_HSR:
    print("row\n")
    for df in row:
        print('df\n')
        print(df['experiment_type'].unique())
        print(df['backend'].unique())



row

df

['Hardware']
['torino' 'brisbane']
df

['Simulation']
['torino' 'brisbane']
df

['Refreshed_Simulation']
['torino' 'brisbane']
df

['Sim and Refreshed']
['torino' 'brisbane']
row

df

['Simulation']
['torino' 'fez' 'marrakesh' 'brisbane']
df

['Hardware']
['torino' 'brisbane']
df

['Refreshed_Simulation']
['torino' 'brisbane']
row

df

['Refreshed_Simulation']
['torino' 'brisbane']
df

['Hardware']
['torino' 'brisbane']
df

['Simulation']
['torino' 'brisbane']
row

df

['Sim and Refreshed']
['torino' 'fez' 'marrakesh' 'brisbane']
df

['Hardware']
['torino' 'brisbane']
